In [1]:
%%bash
export DYLD_LIBRARY_PATH=/Library/PostgreSQL/9.3/lib

In [2]:
import re
import nltk
import psycopg2 as pg
import pandas as pd

from IPython.core.display import HTML
from pandas.io import sql as psql
from nltk.tokenize import RegexpTokenizer
from nltk.stem import RSLPStemmer

ImportError: No module named nltk

In [3]:
pd.set_option('max_colwidth', 300)

## Fetch data from postgres

In [7]:
conn = pg.connect("dbname=ig_news user=postgres password=101010")

### - build games news dataframe (channel "arena")

In [26]:
arena_news_df = psql.read_sql("SELECT url, title, description, body FROM ig_news WHERE channel = 'arena' LIMIT 100;", conn)
arena_news_df['title'] = arena_news_df['title'].apply(lambda t: t.decode('utf-8'))
arena_news_df['description'] = arena_news_df['description'].apply(lambda t: t.decode('utf-8'))
arena_news_df['body'] = arena_news_df['body'].apply(lambda t: t.decode('utf-8'))

# clean channel and ig names from title
arena_news_df['title'] = arena_news_df['title'].apply(lambda t: re.sub(ur' - [\w ]+ - iG$',u'',t,flags=re.UNICODE))

# clean spaces from url
arena_news_df['url'] = arena_news_df['url'].apply(lambda url: url.replace(' ',''))

# join all textual fields
arena_news_df['all'] = arena_news_df['title'] + u' ' + arena_news_df['description'] + u' ' + arena_news_df['body']

arena_news_df

,url,title,description,body,all
0,http://arena.ig.com.br/2014-10-09/os-campeonatos-da-brasil-game-show.html,Os campeonatos da 'Brasil Game Show',Confira os campeonatos que rolam no evento,"O ""Brasil Game Show"" acontece de 09 até 12/10 - no dia 08 abre apenas para imprensa - no Expo Center Norte em São Paulo. Não será vendido ingresso no local, para comprar acesse o site da BGS . A BGS é a maior feira de jogos eletrônicos da América Latina, conta com a presença das maiores ...","Os campeonatos da 'Brasil Game Show' Confira os campeonatos que rolam no evento O ""Brasil Game Show"" acontece de 09 até 12/10 - no dia 08 abre apenas para imprensa - no Expo Center Norte em São Paulo. Não será vendido ingresso no local, para comprar acesse o site da BGS . A BGS é a maior..."
1,http://arena.ig.com.br/2014-09-09/conheca-o-jogo-usado-para-lancar-o-iphone-6.html,Conheça o jogo usado para lançar o iPhone 6,"Vainglory é um multiplayer de batalha, no estilo League of Legends","\n\t \n\t \t\t\t\t\n\t\t Divulgação \n\t\t Desenvolvedor mostrou efeitos de luz do game, rodando a maior resolução em um iPhone 6 \n\t \n A empresa decidiu apresentar o game Vainglory no evento de lançamento do iPhone 6 (e a versão Metal). Tudo como pano de fundo para enaltecer as qualidades grá...","Conheça o jogo usado para lançar o iPhone 6 Vainglory é um multiplayer de batalha, no estilo League of Legends \n\t \n\t \t\t\t\t\n\t\t Divulgação \n\t\t Desenvolvedor mostrou efeitos de luz do game, rodando a maior resolução em um iPhone 6 \n\t \n A empresa decidiu apresentar o game Vainglory n..."
2,http://arena.ig.com.br/2014-09-25/avaliacao-do-fifa-15-por-um-jogador-fanatico.html,Avaliação do FIFA 15 por um jogador fanático,Convidamos um jogador maníaco por FIFA para dizer quais são os pontos positivos e negativos do game,"Aspectos Positivos Estádios A novidade é que o FIFA 15 possui todos os 20 estádios da Premier League (Inglaterra), sendo 12 deles novos no game. O resto dos estádios são dos principais clubes como Real Madrid, Barcelona, Bayern de Munique, Juventus e PSG. Modo Carreira Os olheiros estão mais efi...","Avaliação do FIFA 15 por um jogador fanático Convidamos um jogador maníaco por FIFA para dizer quais são os pontos positivos e negativos do game Aspectos Positivos Estádios A novidade é que o FIFA 15 possui todos os 20 estádios da Premier League (Inglaterra), sendo 12 deles novos no game. O rest..."
3,http://arena.ig.com.br/2014-09-11/paul-mccartney-escreveu-musica-para-destiny-sem-interesses-financeiros.html,Paul McCartney escreveu música para Destiny sem interesses financeiros,"Novo single “Hope for the Future” toca ao final do game, já quando os créditos sobem","\n\t \n\t \t\t\t\t\n\t\t Reprodução \n\t\t Paul aprovou! \n\t \n Não tinha “nenhum cheque envolvido, grande ou de outra forma”, disse Eric Osbourne, gerente de comunidades do game Destiny. “Ele entrou no projeto pela criatividade”, completou Osbourne. Eric revelou também que Paul viu a oportunid...","Paul McCartney escreveu música para Destiny sem interesses financeiros Novo single “Hope for the Future” toca ao final do game, já quando os créditos sobem \n\t \n\t \t\t\t\t\n\t\t Reprodução \n\t\t Paul aprovou! \n\t \n Não tinha “nenhum cheque envolvido, grande ou de outra forma”, disse Eric O..."
4,http://arena.ig.com.br/especiais/infograficos/2014-06-06/infografico-historia-evolucao-dos-controles-joystick-gamepad.html,Infográfico: A evolução dos controles em 40 anos de história dos videogames,Relembramos e detalhamos (quase) todos os controles de videogame já criados pelo homem,"É possível que, ao segurar um controle do Odyssey, o primeiro videogame da história, você sequer saiba como manuseá-lo. Mas o que hoje parece um artefato de design alienígena, no passado ele deu origem a uma cadeia evolutiva que se estendeu pelas próximas quarenta décadas seguintes - e agora cam...","Infográfico: A evolução dos controles em 40 anos de história dos videogames Relembramos e detalhamos (quase) todos os 

In [36]:
def check_duplicates(df):
    for url_1 in df['url']:
        for url_2 in df['url']:
            if url_1 == url_2:
                continue

            if url_1 in url_2:
                print '- url %s in url %s' % (url_1, url_2)

In [46]:
check_duplicates(arena_news_df)

### - build sport news dataframe (channel "esportes")

In [47]:
sport_news_df = psql.read_sql("SELECT url, title, description, body FROM ig_news WHERE channel = 'esporte' LIMIT 100;", conn)
sport_news_df['title'] = sport_news_df['title'].apply(lambda t: t.decode('utf-8'))
sport_news_df['description'] = sport_news_df['description'].apply(lambda t: t.decode('utf-8'))
sport_news_df['body'] = sport_news_df['body'].apply(lambda t: t.decode('utf-8'))

# clean channel and ig names from title
sport_news_df['title'] = sport_news_df['title'].apply(lambda t: re.sub(ur' - [\w ]+ - iG$',u'',t,flags=re.UNICODE))

# clean spaces from url
sport_news_df['url'] = sport_news_df['url'].apply(lambda url: url.replace(' ',''))

# join all textual fields
sport_news_df['all'] = sport_news_df['title'] + u' ' + sport_news_df['description'] + u' ' + sport_news_df['body']

sport_news_df

,url,title,description,body,all
0,http://esporte.ig.com.br/futebol/2014-10-11/ponte-preta-empata-com-o-santa-cruz-em-casa-mas-mantem-lideranca-da-serie-b.html,"Ponte Preta empata com o Santa Cruz em casa, mas mantém liderança da Série B","Os gols do jogo foram marcados por Everton Sena, pelo Santa Cruz, aos 25 minutos da primeira etapa, e pelo atacante Cafú, aos 23 do segundo tempo","Ponte Preta e Santa Cruz se enfrentaram na tarde deste sábado em Campinas. A Ponte queria voltar à liderança da Série B e conseguiu o seu objetivo, mas conseguiu a vitória e apenas empatou com o adversário: 1 a 1. Confira classificação, tabela de jogos, artilharia e notícias da Série B do Brasil...","Ponte Preta empata com o Santa Cruz em casa, mas mantém liderança da Série B Os gols do jogo foram marcados por Everton Sena, pelo Santa Cruz, aos 25 minutos da primeira etapa, e pelo atacante Cafú, aos 23 do segundo tempo Ponte Preta e Santa Cruz se enfrentaram na tarde deste sábado em Campinas..."
1,http://esporte.ig.com.br/futebol/2014-10-11/america-mg-vence-sonha-com-acesso-e-complica-ainda-mais-a-portuguesa.html,"América-MG vence, sonha com acesso e complica ainda mais a Portuguesa","Time de Belo Horizonte chega aos 41 pontos e ainda sonha com o acesso. Já os paulistanos seguem estacionados nos 21 pontos, na lanterna da Série B do Brasileiro","O América-MG não deu bola para o desespero da Portuguesa e venceu por 3 a 1 o jogo realizado neste sábado, no estádio Independência. O revés aumenta o jejum de vitórias da representação rubro-verde para dez jogos, o que coloca a equipe cada vez mais perto do rebaixamento para a Série C do Brasil...","América-MG vence, sonha com acesso e complica ainda mais a Portuguesa Time de Belo Horizonte chega aos 41 pontos e ainda sonha com o acesso. Já os paulistanos seguem estacionados nos 21 pontos, na lanterna da Série B do Brasileiro O América-MG não deu bola para o desespero da Portuguesa e venceu..."
2,http://esporte.ig.com.br/futebol/2014-10-11/jo-nao-aparece-para-treinar-e-vira-desfalque-no-atletico-mg.html,Jô não aparece para treinar e vira desfalque no Atlético-MG,Atacante alegou problemas particulares e caso foi encaminhado para a diretoria. Situação pode se agravar pela reincidência do jogador,"\n\t \n\t \t\t\t\t\n\t\t Miguel Schincariol/Gazeta Press \n\t\t Jô é marcado por Ralf na partida entre Corinthians e Atlético-MG, em Itaquera \n\t \n O atacante Jô não apareceu para treinar no Atlético-MG neste sábado, e está fora do jogo deste domingo, contra o São Paulo, no Independência. O ...",Jô não aparece para treinar e vira desfalque no Atlético-MG Atacante alegou problemas particulares e caso foi encaminhado para a diretoria. Situação pode se agravar pela reincidência do jogador \n\t \n\t \t\t\t\t\n\t\t Miguel Schincariol/Gazeta Press \n\t\t Jô é marcado por Ralf na partida entre...
3,http://esporte.ig.com.br/futebol/2014-10-11/damiao-admite-ano-ruim-vou-me-entregar-ao-maximo-para-melhorar.html,"Damião admite ano ruim: ""Vou me entregar ao máximo para melhorar""","Atacante santista foi criticado pelo ex-presidente, marcou gol da vitória na quarta, mas admite que não rendeu o esperado","\n\t \n\t \t\t\t\t\n\t\t Friedemann Vogel/Getty Images \n\t\t Leandro Damião, atacante do Santos \n\t \n Criticado pelo ex-presidente Luis Álvaro de Oliveira Ribeiro, Leandro Damião deu a resposta dentro de campo na última quinta-feira e marcou um dos gols da vitória do Santos sobre o Bahia. A...","Damião admite ano ruim: ""Vou me entregar ao máximo para melhorar"" Atacante santista foi criticado pelo ex-presidente, marcou gol da vitória na quarta, mas admite que não rendeu o esperado \n\t \n\t \t\t\t\t\n\t\t Friedemann Vogel/Getty Images \n\t\t Leandro Damião, atacante do Santos \n\t \n Cri..."
4,http://esporte.ig.com.br/futebol/2014-10-11/real-madrid-enfrentara-adversario-mexicano-ou-asiatico-no-mundial-de-clubes.html,Real Madrid enfrentará adversário mexicano ou asiático no Mundial de Clubes,"Do outro lado da chave está o 

In [48]:
check_duplicates(sport_news_df)

In [49]:
def put_blank(string):
    blank_str = ''
    for i, shar in enumerate(string):
        if i == len(string) - 1:
            blank_str += shar
        else:
            blank_str += shar + ' '
    return blank_str
put_blank('#div_comentarios')

'# d i v _ c o m e n t a r i o s'

### - build young news dataframe (channel "jovem")

In [50]:
jovem_news_df = psql.read_sql("SELECT url, title, description, body FROM ig_news WHERE channel = 'jovem' LIMIT 100;", conn)
jovem_news_df['title'] = jovem_news_df['title'].apply(lambda t: t.decode('utf-8'))
jovem_news_df['description'] = jovem_news_df['description'].apply(lambda t: t.decode('utf-8'))
jovem_news_df['body'] = jovem_news_df['body'].apply(lambda t: t.decode('utf-8'))

# clean channel and ig names from title
jovem_news_df['title'] = jovem_news_df['title'].apply(lambda t: re.sub(ur' - [\w ]+ - iG$',u'',t,flags=re.UNICODE))

# clean spaces from url
jovem_news_df['url'] = jovem_news_df['url'].apply(lambda url: url.replace(' ',''))

# join all textual fields
jovem_news_df['all'] = jovem_news_df['title'] + u' ' + jovem_news_df['description'] + u' ' + jovem_news_df['body']

jovem_news_df

,url,title,description,body,all
0,http://jovem.ig.com.br/esportesradicais/2013-08-19/adrian-buchan-vence-no-taiti-e-kelly-slater-e-o-novo-lider-do-mundial-de-surfe.html,Adrian Buchan vence no Taiti e Kelly Slater é o novo líder do Mundial de Surfe,"O australiano de 31 anos faturou seu primeiro título na temporada depois de derrotar Slater na final. Com a segunda colocação, o americano reassumiu a liderança do Circuito Mundial","\n\t \n\t \t\t\t\t\n\t\t Getty Images \n\t\t Kelly Slater ficou em segundo na etapa de Teahupoo do Circuito Mundial, mas reassumiu a liderança \n\t \n O australiano Adrian Buchan foi o vencedor da sexta etapa do Circuito Mundial de Surfe , que aconteceu em Teahupoo , Taiti. Ele se sagrou cam...","Adrian Buchan vence no Taiti e Kelly Slater é o novo líder do Mundial de Surfe O australiano de 31 anos faturou seu primeiro título na temporada depois de derrotar Slater na final. Com a segunda colocação, o americano reassumiu a liderança do Circuito Mundial \n\t \n\t \t\t\t\t\n\t\t Getty Image..."
1,http://jovem.ig.com.br/cultura/musica/2014-10-01/banda-de-pop-reggae-chega-ao-topo-da-billboard-o-dinheiro-nao-importa.html,"Banda de pop reggae chega ao topo da Billboard: ""o dinheiro não importa""","Canadenses do Magic! tocaram no Brasil no último sábado (27) e falaram ao iG Jovem sobre seu amor pelo nosso país, o cenário musical e o trabalho do vocalista com Justin Bieber: ""eu o amo""","O Magic! tocou no Brasil no último sábado (27) e alcançou um feito que nunca tinha atingido na carreira. ""Estivemos no topo dos trending topics no Twitter"", contou um entusiasmado Nasri em entrevista concedida em um hotel em São Paulo. ""Já estivemos no topo da Billboard, mas nunca lideramos os t...","Banda de pop reggae chega ao topo da Billboard: ""o dinheiro não importa"" Canadenses do Magic! tocaram no Brasil no último sábado (27) e falaram ao iG Jovem sobre seu amor pelo nosso país, o cenário musical e o trabalho do vocalista com Justin Bieber: ""eu o amo"" O Magic! tocou no Brasil no último..."
2,http://jovem.ig.com.br/cultura/livros/2014-07-15/marvel-surpreende-e-anuncia-que-thor-sera-uma-mulher-em-nova-serie-sobre-o-heroi.html,Marvel surpreende e anuncia que Thor será uma mulher em nova série sobre o herói,"Na história, herói original perde a capacidade de manejar o martelo mágico e passa seus poderes para uma garota","Novidade bomba para os fãs de quadrinhos! A Marvel Comics anunciou na terça-feira (15) que Thor , um de seus principais personagens (interpretado no cinema por Chris Hemsworth), será substituído por uma mulher na nova série sobre o Deus do Trovão. A revelação foi feita durante o programa matina...","Marvel surpreende e anuncia que Thor será uma mulher em nova série sobre o herói Na história, herói original perde a capacidade de manejar o martelo mágico e passa seus poderes para uma garota Novidade bomba para os fãs de quadrinhos! A Marvel Comics anunciou na terça-feira (15) que Thor , um d..."
3,http://jovem.ig.com.br/cultura/filmes/2014-07-15/comic-con-deixa-jogos-vorazes-doctor-who-e-divergente-de-fora.html,"Comic-Con deixa Jogos Vorazes, Doctor Who e Divergente de fora","Atrações do evento, o principal do mundo quando o assunto é cultura pop, foram anunciadas nessa segunda-feira (14). Apesar da expectativa dos fãs, títulos importantes como ""Quarteto Fantástico"" não aparecem na agenda oficial da feira","\n\t \n\t \t\t\t\t\n\t\t Divulgação \n\t\t ""Jogos Vorazes - A Esperança"" não estará na programação da feira \n\t \n A Comic-Son International, principal evento do mundo de cultura pop, começa na próxima semana em San Diego, e para a decepção de alguns fãs, muitos títulos importantes do cinema e ...","Comic-Con deixa Jogos Vorazes, Doctor Who e Divergente de fora Atrações do evento, o principal do mundo quando o assunto é cultura pop, foram anunciadas nessa segunda-feira (14). Apesar da expectativa dos fãs, títulos importantes como ""Quarteto Fantástico"" não aparecem na agenda oficial da feira..."

In [51]:
check_duplicates(jovem_news_df)

## Text cleaning

### - Text to lower

In [52]:
arena_news_df['all'] = arena_news_df['all'].apply(lambda t: t.lower())
sport_news_df['all'] = sport_news_df['all'].apply(lambda t: t.lower())
jovem_news_df['all'] = jovem_news_df['all'].apply(lambda t: t.lower())

pd.DataFrame(arena_news_df['all']).head()

,all
0,"os campeonatos da 'brasil game show' confira os campeonatos que rolam no evento o ""brasil game show"" acontece de 09 até 12/10 - no dia 08 abre apenas para imprensa - no expo center norte em são paulo. não será vendido ingresso no local, para comprar acesse o site da bgs . a bgs é a maior..."
1,"conheça o jogo usado para lançar o iphone 6 vainglory é um multiplayer de batalha, no estilo league of legends \n\t \n\t \t\t\t\t\n\t\t divulgação \n\t\t desenvolvedor mostrou efeitos de luz do game, rodando a maior resolução em um iphone 6 \n\t \n a empresa decidiu apresentar o game vainglory n..."
2,"avaliação do fifa 15 por um jogador fanático convidamos um jogador maníaco por fifa para dizer quais são os pontos positivos e negativos do game aspectos positivos estádios a novidade é que o fifa 15 possui todos os 20 estádios da premier league (inglaterra), sendo 12 deles novos no game. o rest..."
3,"paul mccartney escreveu música para destiny sem interesses financeiros novo single “hope for the future” toca ao final do game, já quando os créditos sobem \n\t \n\t \t\t\t\t\n\t\t reprodução \n\t\t paul aprovou! \n\t \n não tinha “nenhum cheque envolvido, grande ou de outra forma”, disse eric o..."
4,"infográfico: a evolução dos controles em 40 anos de história dos videogames relembramos e detalhamos (quase) todos os controles de videogame já criados pelo homem é possível que, ao segurar um controle do odyssey, o primeiro videogame da história, você sequer saiba como manuseá-lo. mas o que hoj..."


### - Cleaning puctuation

In [53]:
def clean_punct(text):
    cleaned = re.sub(ur'[\?!~#,;.\'"“”´`:\(\)&\*]',u'',text,flags=re.UNICODE)
    cleaned = re.sub(ur'[\\\/\n\t\r]',u' ',cleaned,flags=re.UNICODE)
    return re.sub(ur'( )+[_-]+( )+',u' ',cleaned,flags=re.UNICODE) # replace isolated - or _

arena_news_df['all'] = arena_news_df['all'].apply(clean_punct)
sport_news_df['all'] = sport_news_df['all'].apply(clean_punct)
jovem_news_df['all'] = jovem_news_df['all'].apply(clean_punct)

pd.DataFrame(arena_news_df['all']).head()

,all
0,os campeonatos da brasil game show confira os campeonatos que rolam no evento o brasil game show acontece de 09 até 12 10 - no dia 08 abre apenas para imprensa no expo center norte em são paulo não será vendido ingresso no local para comprar acesse o site da bgs a bgs é a maior feira de...
1,conheça o jogo usado para lançar o iphone 6 vainglory é um multiplayer de batalha no estilo league of legends divulgação desenvolvedor mostrou efeitos de luz do game rodando a maior resolução em um iphone 6 a empresa decidiu apresentar o game vainglory no evento de lançame...
2,avaliação do fifa 15 por um jogador fanático convidamos um jogador maníaco por fifa para dizer quais são os pontos positivos e negativos do game aspectos positivos estádios a novidade é que o fifa 15 possui todos os 20 estádios da premier league inglaterra sendo 12 deles novos no game o resto do...
3,paul mccartney escreveu música para destiny sem interesses financeiros novo single hope for the future toca ao final do game já quando os créditos sobem reprodução paul aprovou não tinha nenhum cheque envolvido grande ou de outra forma disse eric osbourne gerente de comuni...
4,infográfico a evolução dos controles em 40 anos de história dos videogames relembramos e detalhamos quase todos os controles de videogame já criados pelo homem é possível que ao segurar um controle do odyssey o primeiro videogame da história você sequer saiba como manuseá-lo mas o que hoje parec...


In [54]:
pd.DataFrame(sport_news_df['all']).head()

,all
0,ponte preta empata com o santa cruz em casa mas mantém liderança da série b os gols do jogo foram marcados por everton sena pelo santa cruz aos 25 minutos da primeira etapa e pelo atacante cafú aos 23 do segundo tempo ponte preta e santa cruz se enfrentaram na tarde deste sábado em campinas a po...
1,américa-mg vence sonha com acesso e complica ainda mais a portuguesa time de belo horizonte chega aos 41 pontos e ainda sonha com o acesso já os paulistanos seguem estacionados nos 21 pontos na lanterna da série b do brasileiro o américa-mg não deu bola para o desespero da portuguesa e venceu po...
2,jô não aparece para treinar e vira desfalque no atlético-mg atacante alegou problemas particulares e caso foi encaminhado para a diretoria situação pode se agravar pela reincidência do jogador miguel schincariol gazeta press jô é marcado por ralf na partida entre corinthians e ...
3,damião admite ano ruim vou me entregar ao máximo para melhorar atacante santista foi criticado pelo ex-presidente marcou gol da vitória na quarta mas admite que não rendeu o esperado friedemann vogel getty images leandro damião atacante do santos criticado pelo ex-presiden...
4,real madrid enfrentará adversário mexicano ou asiático no mundial de clubes do outro lado da chave está o san lorenzo da argentina que tem quatro possíveis adversários conforme o sorteio atual campeão europeu o real madrid irá enfrentar o mexicano cruz azul ou o campeão asiático que ainda está...


### - Clean monetary stuff

In [55]:
def clean_monetary(text):
    return re.sub(ur'(r){0,1}\$[ ]{0,1}\d+([,.]\d+)*',u'monetáriointerno',text,flags=re.UNICODE)

arena_news_df['all'] = arena_news_df['all'].apply(clean_monetary)
sport_news_df['all'] = sport_news_df['all'].apply(clean_monetary)
jovem_news_df['all'] = jovem_news_df['all'].apply(clean_monetary)

pd.DataFrame(arena_news_df['all']).head()

,all
0,os campeonatos da brasil game show confira os campeonatos que rolam no evento o brasil game show acontece de 09 até 12 10 - no dia 08 abre apenas para imprensa no expo center norte em são paulo não será vendido ingresso no local para comprar acesse o site da bgs a bgs é a maior feira de...
1,conheça o jogo usado para lançar o iphone 6 vainglory é um multiplayer de batalha no estilo league of legends divulgação desenvolvedor mostrou efeitos de luz do game rodando a maior resolução em um iphone 6 a empresa decidiu apresentar o game vainglory no evento de lançame...
2,avaliação do fifa 15 por um jogador fanático convidamos um jogador maníaco por fifa para dizer quais são os pontos positivos e negativos do game aspectos positivos estádios a novidade é que o fifa 15 possui todos os 20 estádios da premier league inglaterra sendo 12 deles novos no game o resto do...
3,paul mccartney escreveu música para destiny sem interesses financeiros novo single hope for the future toca ao final do game já quando os créditos sobem reprodução paul aprovou não tinha nenhum cheque envolvido grande ou de outra forma disse eric osbourne gerente de comuni...
4,infográfico a evolução dos controles em 40 anos de história dos videogames relembramos e detalhamos quase todos os controles de videogame já criados pelo homem é possível que ao segurar um controle do odyssey o primeiro videogame da história você sequer saiba como manuseá-lo mas o que hoje parec...


In [56]:
pd.DataFrame(sport_news_df['all']).head()

,all
0,ponte preta empata com o santa cruz em casa mas mantém liderança da série b os gols do jogo foram marcados por everton sena pelo santa cruz aos 25 minutos da primeira etapa e pelo atacante cafú aos 23 do segundo tempo ponte preta e santa cruz se enfrentaram na tarde deste sábado em campinas a po...
1,américa-mg vence sonha com acesso e complica ainda mais a portuguesa time de belo horizonte chega aos 41 pontos e ainda sonha com o acesso já os paulistanos seguem estacionados nos 21 pontos na lanterna da série b do brasileiro o américa-mg não deu bola para o desespero da portuguesa e venceu po...
2,jô não aparece para treinar e vira desfalque no atlético-mg atacante alegou problemas particulares e caso foi encaminhado para a diretoria situação pode se agravar pela reincidência do jogador miguel schincariol gazeta press jô é marcado por ralf na partida entre corinthians e ...
3,damião admite ano ruim vou me entregar ao máximo para melhorar atacante santista foi criticado pelo ex-presidente marcou gol da vitória na quarta mas admite que não rendeu o esperado friedemann vogel getty images leandro damião atacante do santos criticado pelo ex-presiden...
4,real madrid enfrentará adversário mexicano ou asiático no mundial de clubes do outro lado da chave está o san lorenzo da argentina que tem quatro possíveis adversários conforme o sorteio atual campeão europeu o real madrid irá enfrentar o mexicano cruz azul ou o campeão asiático que ainda está...


### - Clean numerals

In [57]:
def clean_numerals(text):
    return re.sub(ur'(\d+[ºª])|(\d+((th)|(rd)|(st)))',u'',text,flags=re.UNICODE)

arena_news_df['all'] = arena_news_df['all'].apply(clean_numerals)
sport_news_df['all'] = sport_news_df['all'].apply(clean_numerals)
jovem_news_df['all'] = jovem_news_df['all'].apply(clean_numerals)

pd.DataFrame(arena_news_df['all']).head()

,all
0,os campeonatos da brasil game show confira os campeonatos que rolam no evento o brasil game show acontece de 09 até 12 10 - no dia 08 abre apenas para imprensa no expo center norte em são paulo não será vendido ingresso no local para comprar acesse o site da bgs a bgs é a maior feira de...
1,conheça o jogo usado para lançar o iphone 6 vainglory é um multiplayer de batalha no estilo league of legends divulgação desenvolvedor mostrou efeitos de luz do game rodando a maior resolução em um iphone 6 a empresa decidiu apresentar o game vainglory no evento de lançame...
2,avaliação do fifa 15 por um jogador fanático convidamos um jogador maníaco por fifa para dizer quais são os pontos positivos e negativos do game aspectos positivos estádios a novidade é que o fifa 15 possui todos os 20 estádios da premier league inglaterra sendo 12 deles novos no game o resto do...
3,paul mccartney escreveu música para destiny sem interesses financeiros novo single hope for the future toca ao final do game já quando os créditos sobem reprodução paul aprovou não tinha nenhum cheque envolvido grande ou de outra forma disse eric osbourne gerente de comuni...
4,infográfico a evolução dos controles em 40 anos de história dos videogames relembramos e detalhamos quase todos os controles de videogame já criados pelo homem é possível que ao segurar um controle do odyssey o primeiro videogame da história você sequer saiba como manuseá-lo mas o que hoje parec...


### - Clean digits

In [58]:
def clean_digits(text):
    return re.sub(ur'[0-9]+',u'',text,flags=re.UNICODE)

arena_news_df['all'] = arena_news_df['all'].apply(clean_digits)
sport_news_df['all'] = sport_news_df['all'].apply(clean_digits)
jovem_news_df['all'] = jovem_news_df['all'].apply(clean_digits)

pd.DataFrame(arena_news_df['all']).head()

,all
0,os campeonatos da brasil game show confira os campeonatos que rolam no evento o brasil game show acontece de até - no dia abre apenas para imprensa no expo center norte em são paulo não será vendido ingresso no local para comprar acesse o site da bgs a bgs é a maior feira de jogos e...
1,conheça o jogo usado para lançar o iphone vainglory é um multiplayer de batalha no estilo league of legends divulgação desenvolvedor mostrou efeitos de luz do game rodando a maior resolução em um iphone a empresa decidiu apresentar o game vainglory no evento de lançament...
2,avaliação do fifa por um jogador fanático convidamos um jogador maníaco por fifa para dizer quais são os pontos positivos e negativos do game aspectos positivos estádios a novidade é que o fifa possui todos os estádios da premier league inglaterra sendo deles novos no game o resto dos estádi...
3,paul mccartney escreveu música para destiny sem interesses financeiros novo single hope for the future toca ao final do game já quando os créditos sobem reprodução paul aprovou não tinha nenhum cheque envolvido grande ou de outra forma disse eric osbourne gerente de comuni...
4,infográfico a evolução dos controles em anos de história dos videogames relembramos e detalhamos quase todos os controles de videogame já criados pelo homem é possível que ao segurar um controle do odyssey o primeiro videogame da história você sequer saiba como manuseá-lo mas o que hoje parece ...


### - Clean web stuff

In [59]:
def clean_web(text):
    return re.sub(ur'(www\w*)',u'',text,flags=re.UNICODE)

arena_news_df['all'] = arena_news_df['all'].apply(clean_web)
sport_news_df['all'] = sport_news_df['all'].apply(clean_web)
jovem_news_df['all'] = jovem_news_df['all'].apply(clean_web)

pd.DataFrame(sport_news_df['all']).head()

,all
0,ponte preta empata com o santa cruz em casa mas mantém liderança da série b os gols do jogo foram marcados por everton sena pelo santa cruz aos minutos da primeira etapa e pelo atacante cafú aos do segundo tempo ponte preta e santa cruz se enfrentaram na tarde deste sábado em campinas a ponte ...
1,américa-mg vence sonha com acesso e complica ainda mais a portuguesa time de belo horizonte chega aos pontos e ainda sonha com o acesso já os paulistanos seguem estacionados nos pontos na lanterna da série b do brasileiro o américa-mg não deu bola para o desespero da portuguesa e venceu por a...
2,jô não aparece para treinar e vira desfalque no atlético-mg atacante alegou problemas particulares e caso foi encaminhado para a diretoria situação pode se agravar pela reincidência do jogador miguel schincariol gazeta press jô é marcado por ralf na partida entre corinthians e ...
3,damião admite ano ruim vou me entregar ao máximo para melhorar atacante santista foi criticado pelo ex-presidente marcou gol da vitória na quarta mas admite que não rendeu o esperado friedemann vogel getty images leandro damião atacante do santos criticado pelo ex-presiden...
4,real madrid enfrentará adversário mexicano ou asiático no mundial de clubes do outro lado da chave está o san lorenzo da argentina que tem quatro possíveis adversários conforme o sorteio atual campeão europeu o real madrid irá enfrentar o mexicano cruz azul ou o campeão asiático que ainda está...


### - Tokenizing

In [60]:
tokenizer = RegexpTokenizer(u"(\w+)")

arena_news_df['all'] = arena_news_df['all'].apply(lambda text: tokenizer.tokenize(text))
sport_news_df['all'] = sport_news_df['all'].apply(lambda text: tokenizer.tokenize(text))
jovem_news_df['all'] = jovem_news_df['all'].apply(lambda text: tokenizer.tokenize(text))

pd.DataFrame(arena_news_df['all']).head()

,all
0,"[os, campeonatos, da, brasil, game, show, confira, os, campeonatos, que, rolam, no, evento, o, brasil, game, show, acontece, de, até, no, dia, abre, apenas, para, imprensa, no, expo, center, norte, em, são, paulo, não, será, vendido, ingresso, no, local, para, comprar, acesse, o, site, da, bgs, ..."
1,"[conheça, o, jogo, usado, para, lançar, o, iphone, vainglory, é, um, multiplayer, de, batalha, no, estilo, league, of, legends, divulgação, desenvolvedor, mostrou, efeitos, de, luz, do, game, rodando, a, maior, resolução, em, um, iphone, a, empresa, decidiu, apresentar, o, game, vainglory, no, e..."
2,"[avaliação, do, fifa, por, um, jogador, fanático, convidamos, um, jogador, maníaco, por, fifa, para, dizer, quais, são, os, pontos, positivos, e, negativos, do, game, aspectos, positivos, estádios, a, novidade, é, que, o, fifa, possui, todos, os, estádios, da, premier, league, inglaterra, sendo,..."
3,"[paul, mccartney, escreveu, música, para, destiny, sem, interesses, financeiros, novo, single, hope, for, the, future, toca, ao, final, do, game, já, quando, os, créditos, sobem, reprodução, paul, aprovou, não, tinha, nenhum, cheque, envolvido, grande, ou, de, outra, forma, disse, eric, osbourne..."
4,"[infográfico, a, evolução, dos, controles, em, anos, de, história, dos, videogames, relembramos, e, detalhamos, quase, todos, os, controles, de, videogame, já, criados, pelo, homem, é, possível, que, ao, segurar, um, controle, do, odyssey, o, primeiro, videogame, da, história, você, sequer, saib..."


### - Filtering stopwords

In [61]:
stopwords = nltk.corpus.stopwords.words('portuguese')

stopwords = [w.decode('utf-8') for w in stopwords]
stopwords.extend([u'á',u'é',u'ó',u'i',u'u'])
stopwords.extend([u'leia',u'lendo',u'twitter',u'facebook',u'mais',u'divulgação',u'agnews',
                  u'ler',u'ig',u'reprodução',u'siga',u'curta',u'images',u'imagens',u'veja',
                  u'tambem',u'também',u'saiba',u'informações',u'thinkstock',u'getty',u'photos',
                  u'foto',u'fotos',u'ap',u'gettyimages',u'infográfico',u'aí']) 

new_stopwords = [w.rstrip('\n').decode('utf-8') for w in open('stopwords.txt')]
stopwords.extend(new_stopwords)
stopwords = set(stopwords)

print ', '.join(stopwords)

nenhum, essas, sejam, algumas, ver, feitas, deviam, tu, vez, ti, te, deveriam, dela, á, dele, éramos, houveria, facebook, tinham, disto, somos, diz, siga, tem, nesta, ter, teu, nem, sua, fôssemos, primeiros, feita, deles, feito, leia, meu, ler, à, grandes, thinkstock, vir, outra, outro, estávamos, estive, tém, todas, seja, tudo, pequena, pouca, pequeno, pouco, devendo, me, você, porém, só, muitos, minhas, era, tuas, mesmos, perante, fossem, tampouco, seremos, daqueles, deveria, seriam, saiba, vindo, delas, mas, pois, vocês, primeiro, houve, aos, numa, não, estivéramos, a, lhe, deve, isso, si, grande, ainda, hajamos, sejamos, próprias, sob, nestas, mais, la, lo, nossos, imagens, devem, dever, tivessem, alguém, das, pelas, nos, através, até, sido, serão, em, nossa, toda, eu, todo, nosso, gettyimages, lá, nunca, hajam, fazer, destes, foto, e', formos, às, terão, que, amplas, terá, estejamos, tivesse, está, teríamos, cada, deverá, enquanto, contra, dessas, para, quando, talvez, estavam, qu

In [62]:
def filter_stopwords(tokens):
    return [w for w in tokens if w not in stopwords]

arena_news_df['all'] = arena_news_df['all'].apply(filter_stopwords)
sport_news_df['all'] = sport_news_df['all'].apply(filter_stopwords)
jovem_news_df['all'] = jovem_news_df['all'].apply(filter_stopwords)

pd.DataFrame(arena_news_df['all']).head()

,all
0,"[campeonatos, brasil, game, show, confira, campeonatos, rolam, evento, brasil, game, show, acontece, dia, abre, apenas, imprensa, expo, center, norte, paulo, vendido, ingresso, local, comprar, acesse, site, bgs, bgs, maior, feira, jogos, eletrônicos, américa, latina, conta, presença, maiores, em..."
1,"[conheça, jogo, usado, lançar, iphone, vainglory, multiplayer, batalha, estilo, league, of, legends, desenvolvedor, mostrou, efeitos, luz, game, rodando, maior, resolução, iphone, empresa, decidiu, apresentar, game, vainglory, evento, lançamento, iphone, versão, metal, pano, fundo, enaltecer, qu..."
2,"[avaliação, fifa, jogador, fanático, convidamos, jogador, maníaco, fifa, dizer, pontos, positivos, negativos, game, aspectos, positivos, estádios, novidade, fifa, possui, estádios, premier, league, inglaterra, novos, game, resto, estádios, principais, clubes, real, madrid, barcelona, bayern, mun..."
3,"[paul, mccartney, escreveu, música, destiny, interesses, financeiros, novo, single, hope, the, future, toca, final, game, créditos, sobem, paul, aprovou, cheque, envolvido, forma, eric, osbourne, gerente, comunidades, game, destiny, entrou, projeto, criatividade, completou, osbourne, eric, revel..."
4,"[evolução, controles, anos, história, videogames, relembramos, detalhamos, quase, controles, videogame, criados, homem, possível, segurar, controle, odyssey, videogame, história, sequer, manuseá, hoje, parece, artefato, design, alienígena, passado, deu, origem, cadeia, evolutiva, estendeu, próxi..."


### - Stemming

In [63]:
def stem(tokens):
    stemmer = RSLPStemmer()
    return [stemmer.stem(w) for w in tokens]

arena_news_stem_df = arena_news_df.copy()
sport_news_stem_df = sport_news_df.copy()
jovem_news_stem_df = jovem_news_df.copy()

arena_news_stem_df['all'] = arena_news_stem_df['all'].apply(stem)
sport_news_stem_df['all'] = sport_news_stem_df['all'].apply(stem)
jovem_news_stem_df = jovem_news_df.copy()

pd.DataFrame(jovem_news_stem_df['all']).head()

,all
0,"[adrian, buchan, vence, taiti, kelly, slater, novo, líder, mundial, surfe, australiano, anos, faturou, título, temporada, derrotar, slater, final, segunda, colocação, americano, reassumiu, liderança, circuito, mundial, kelly, slater, ficou, segundo, etapa, teahupoo, circuito, mundial, reassumiu,..."
1,"[banda, pop, reggae, chega, topo, billboard, dinheiro, importa, canadenses, magic, tocaram, brasil, sábado, falaram, jovem, amor, país, cenário, musical, trabalho, vocalista, justin, bieber, amo, magic, tocou, brasil, sábado, alcançou, atingido, carreira, topo, trending, topics, contou, entusias..."
2,"[marvel, surpreende, anuncia, thor, mulher, nova, série, herói, história, herói, original, perde, capacidade, manejar, martelo, mágico, passa, poderes, garota, novidade, bomba, fãs, quadrinhos, marvel, comics, anunciou, terça, feira, thor, principais, personagens, interpretado, cinema, chris, he..."
3,"[comic, con, deixa, jogos, vorazes, doctor, who, divergente, atrações, evento, principal, mundo, assunto, cultura, pop, anunciadas, segunda, feira, apesar, expectativa, fãs, títulos, importantes, quarteto, fantástico, aparecem, agenda, oficial, feira, jogos, vorazes, esperança, estará, programaç..."
4,"[trio, transforma, física, diversão, ciência, casa, novo, programa, nat, geo, ensina, matéria, temida, colégio, ajuda, explosões, mochila, foguete, vôo, balões, edu, cesar, fotoarena, ideia, pegar, assunto, dia, dia, explorar, máximo, explicam, físicos, gerson, daniel, wilson, aprender, física, ..."


### - Transform back to text

In [64]:
arena_news_df['all'] = arena_news_df['all'].apply(lambda tokens: u' '.join(tokens))
sport_news_df['all'] = sport_news_df['all'].apply(lambda tokens: u' '.join(tokens))
jovem_news_df['all'] = jovem_news_df['all'].apply(lambda tokens: u' '.join(tokens))

arena_news_stem_df['all'] = arena_news_stem_df['all'].apply(lambda tokens: u' '.join(tokens))
sport_news_stem_df['all'] = sport_news_stem_df['all'].apply(lambda tokens: u' '.join(tokens))
jovem_news_stem_df['all'] = jovem_news_stem_df['all'].apply(lambda tokens: u' '.join(tokens))

pd.DataFrame(arena_news_df['all']).head()

,all
0,campeonatos brasil game show confira campeonatos rolam evento brasil game show acontece dia abre apenas imprensa expo center norte paulo vendido ingresso local comprar acesse site bgs bgs maior feira jogos eletrônicos américa latina conta presença maiores empresas games mundo empresas feira torn...
1,conheça jogo usado lançar iphone vainglory multiplayer batalha estilo league of legends desenvolvedor mostrou efeitos luz game rodando maior resolução iphone empresa decidiu apresentar game vainglory evento lançamento iphone versão metal pano fundo enaltecer qualidades gráficas api gráfica ios n...
2,avaliação fifa jogador fanático convidamos jogador maníaco fifa dizer pontos positivos negativos game aspectos positivos estádios novidade fifa possui estádios premier league inglaterra novos game resto estádios principais clubes real madrid barcelona bayern munique juventus psg modo carreira ol...
3,paul mccartney escreveu música destiny interesses financeiros novo single hope the future toca final game créditos sobem paul aprovou cheque envolvido forma eric osbourne gerente comunidades game destiny entrou projeto criatividade completou osbourne eric revelou paul viu oportunidade alcançar a...
4,evolução controles anos história videogames relembramos detalhamos quase controles videogame criados homem possível segurar controle odyssey videogame história sequer manuseá hoje parece artefato design alienígena passado deu origem cadeia evolutiva estendeu próximas quarenta décadas seguintes c...


## Save datasets

In [65]:
arena_news_stem_df.to_pickle('arena_news_stem_df.pkl')
sport_news_stem_df.to_pickle('sport_news_stem_df.pkl')
jovem_news_stem_df.to_pickle('jovem_news_stem_df.pkl')
arena_news_df.to_pickle('arena_news_df.pkl')
sport_news_df.to_pickle('sport_news_df.pkl')
jovem_news_df.to_pickle('jovem_news_df.pkl')